#  Detecting design elements from dynamic information

Using static information is not enough to capture design elements in a program, particularly for statically-typed languages such as Python.


### Tracing executions in Python

Fortunately, it is naturally able to capture dynamic information in Python as follows:

In [6]:
from types import FrameType, TracebackType
from typing import Any, Optional, Callable

def traceit(frame: FrameType, event: str, arg: Any) -> Optional[Callable]:
    print(event, frame.f_lineno, frame.f_code.co_name, frame.f_locals)
    return traceit

Now we can trace the execution of the following simple program.

In [7]:
def remove_html_markup(s):
    tag = False
    quote = False
    out = ""

    for c in s:
        if c == '<' and not quote:
            tag = True
        elif c == '>' and not quote:
            tag = False
        elif c == '"' or c == "'" and tag:
            quote = not quote
        elif not tag:
            out = out + c

    return out

In [8]:
import sys

def remove_html_markup_traced(s):
    sys.settrace(traceit)
    ret = remove_html_markup(s)
    sys.settrace(None)
    return ret

In [9]:
remove_html_markup_traced('xyz')

call 1 remove_html_markup {'s': 'xyz'}
line 2 remove_html_markup {'s': 'xyz'}
line 3 remove_html_markup {'s': 'xyz', 'tag': False}
line 4 remove_html_markup {'s': 'xyz', 'tag': False, 'quote': False}
line 6 remove_html_markup {'s': 'xyz', 'tag': False, 'quote': False, 'out': ''}
line 7 remove_html_markup {'s': 'xyz', 'tag': False, 'quote': False, 'out': '', 'c': 'x'}
line 9 remove_html_markup {'s': 'xyz', 'tag': False, 'quote': False, 'out': '', 'c': 'x'}
line 11 remove_html_markup {'s': 'xyz', 'tag': False, 'quote': False, 'out': '', 'c': 'x'}
line 13 remove_html_markup {'s': 'xyz', 'tag': False, 'quote': False, 'out': '', 'c': 'x'}
line 14 remove_html_markup {'s': 'xyz', 'tag': False, 'quote': False, 'out': '', 'c': 'x'}
line 6 remove_html_markup {'s': 'xyz', 'tag': False, 'quote': False, 'out': 'x', 'c': 'x'}
line 7 remove_html_markup {'s': 'xyz', 'tag': False, 'quote': False, 'out': 'x', 'c': 'y'}
line 9 remove_html_markup {'s': 'xyz', 'tag': False, 'quote': False, 'out': 'x', 'c':

'xyz'

We can focus on a specific part of a program.


In [13]:
def traceit2(frame: FrameType, event: str, arg: Any) -> Optional[Callable]:
    if 'c' in frame.f_locals:
        value_of_c = frame.f_locals['c']
        print(f"{frame.f_lineno:} c = {repr(value_of_c)}")
    else:
        print(f"{frame.f_lineno:} c is undefined")

    return traceit

In [15]:
def remove_html_markup_traced2(s):
    sys.settrace(traceit2)
    ret = remove_html_markup(s)
    sys.settrace(None)
    return ret

In [16]:
remove_html_markup_traced2('xyz')

1 c is undefined
2 c is undefined
3 c is undefined
4 c is undefined
6 c is undefined
7 c = 'x'
9 c = 'x'
11 c = 'x'
13 c = 'x'
14 c = 'x'
6 c = 'x'
7 c = 'y'
9 c = 'y'
11 c = 'y'
13 c = 'y'
14 c = 'y'
6 c = 'y'
7 c = 'z'
9 c = 'z'
11 c = 'z'
13 c = 'z'
14 c = 'z'
6 c = 'z'
16 c = 'z'
16 c = 'z'


'xyz'

### Systematic tracing of program execution

In [29]:
from utils import StackInspector
from typing import TextIO, Type

class Tracer(StackInspector):
    """A class for tracing a piece of code. Use as `with Tracer(): block()`"""

    def __init__(self, *, file: TextIO = sys.stdout) -> None:
        """Trace a block of code, sending logs to `file` (default: stdout)"""
        self.original_trace_function: Optional[Callable] = None
        self.file = file

    def traceit(self, frame: FrameType, event: str, arg: Any) -> None:
        """Tracing function. To be overridden in subclasses."""
        self.log(event, frame.f_lineno, frame.f_code.co_name, frame.f_locals)

    def _traceit(self, frame: FrameType, event: str, arg: Any) -> Optional[Callable]:
        """Internal tracing function."""
        if self.our_frame(frame):
            # Do not trace our own methods
            pass
        else:
            self.traceit(frame, event, arg)
        return self._traceit

    def log(self, *objects: Any, 
            sep: str = ' ', end: str = '\n', 
            flush: bool = True) -> None:
        """
        Like `print()`, but always sending to `file` given at initialization,
        and flushing by default.
        """
        print(*objects, sep=sep, end=end, file=self.file, flush=flush)

    def __enter__(self) -> Any:
        """Called at begin of `with` block. Turn tracing on."""
        self.original_trace_function = sys.gettrace()
        sys.settrace(self._traceit)

        # This extra line also enables tracing for the current block
        # inspect.currentframe().f_back.f_trace = self._traceit
        return self

    def __exit__(self, exc_tp: Type, exc_value: BaseException, 
                 exc_traceback: TracebackType) -> Optional[bool]:
        """
        Called at end of `with` block. Turn tracing off.
        Return `None` if ok, not `None` if internal error.
        """
        sys.settrace(self.original_trace_function)

        # Note: we must return a non-True value here,
        # such that we re-raise all exceptions
        if self.is_internal_error(exc_tp, exc_value, exc_traceback):
            return False  # internal error
        else:
            return None  # all ok

In [30]:
with Tracer():
    remove_html_markup("abc")

call 1 remove_html_markup {'s': 'abc'}
line 2 remove_html_markup {'s': 'abc'}
line 3 remove_html_markup {'s': 'abc', 'tag': False}
line 4 remove_html_markup {'s': 'abc', 'tag': False, 'quote': False}
line 6 remove_html_markup {'s': 'abc', 'tag': False, 'quote': False, 'out': ''}
line 7 remove_html_markup {'s': 'abc', 'tag': False, 'quote': False, 'out': '', 'c': 'a'}
line 9 remove_html_markup {'s': 'abc', 'tag': False, 'quote': False, 'out': '', 'c': 'a'}
line 11 remove_html_markup {'s': 'abc', 'tag': False, 'quote': False, 'out': '', 'c': 'a'}
line 13 remove_html_markup {'s': 'abc', 'tag': False, 'quote': False, 'out': '', 'c': 'a'}
line 14 remove_html_markup {'s': 'abc', 'tag': False, 'quote': False, 'out': '', 'c': 'a'}
line 6 remove_html_markup {'s': 'abc', 'tag': False, 'quote': False, 'out': 'a', 'c': 'a'}
line 7 remove_html_markup {'s': 'abc', 'tag': False, 'quote': False, 'out': 'a', 'c': 'b'}
line 9 remove_html_markup {'s': 'abc', 'tag': False, 'quote': False, 'out': 'a', 'c':